In [1]:
from gnuradio import blocks
from gnuradio import filter
import pmt
from gnuradio import gr
from gnuradio.fft import window
from gnuradio import filter
import sys
import signal
from gnuradio import uhd
import time
import random
from gnuradio import analog
from gnuradio.filter import firdes
# from sklearn import preprocessing
import pickle

In [2]:
DVB_link =  '/home/leesiho/matlab/lab/preamble_signal/p1_SISO_original.dat'
WIFI_link = '/home/leesiho/matlab/lab/preamble_signal/heSU_bw20_original.dat'
Radar_vector = 8*([1]*100 + [0]*100)
# temp_index = '4'
on_off_factor = [0,0,1] # radar / dvb / wifi 
test_file_factor = [1,0] # test / valid 
sub_idx_from = 1
sub_idx_to = 9
n_ans = 100

if on_off_factor[0] == 1:
    gain_value = 5 # radar
if on_off_factor[1] == 1:
    gain_value = 13 # dvb
if on_off_factor[2] == 1:
    gain_value = 16 # wifi
    
if test_file_factor[0] == 1:
    path = 'test_data' # test_1
    label_name = 'ANS_2_0_test.dat'
if test_file_factor[1] == 1:
    path = 'test_test_data' # test_0    
    label_name = 'ANS_2_0_valid.dat'
    
  

In [3]:
class problem2(gr.top_block):

    def __init__(self):
        gr.top_block.__init__(self, "Problem2", catch_exceptions=True)

        ##################################################
        # Variables
        ##################################################
        self.offset1 = offset1 = 0
        self.offset2 = offset2 = 0
        self.offset3 = offset3 = 0
        self.head = head = 1000000
        self.skip_head = skip_head = int(1e6)

        if on_off_factor[0] == 1:
            self.file_path = file_path = '/home/leesiho/matlab/lab/2/'+path+'/radar/radar_test'+temp_index+'.bin'
        if on_off_factor[1] == 1:
            self.file_path = file_path = '/home/leesiho/matlab/lab/2/'+path+'/dvb/dvb_test'+temp_index+'.bin'
        if on_off_factor[2] == 1:
            self.file_path = file_path = '/home/leesiho/matlab/lab/2/'+path+'/wifi/wifi_test'+temp_index+'.bin'

                    
        self.Tx_rate1 = Tx_rate1 = 1e6
        self.Tx_rate2 = Tx_rate2 = 1e6
        self.Tx_rate3 = Tx_rate3 = 1e6
        self.Tx_freq1 = Tx_freq1 = 1.2e9 
        self.Tx_freq2 = Tx_freq2 = 1.2e9 
        self.Tx_freq3 = Tx_freq3 = 1.2e9
        self.Rx_rate = Rx_rate = 10e6
        self.Rx_freq = Rx_freq = 1.2e9
        self.vector_source = vector_source = Radar_vector
        self.DVB_2T_source = DVB_2T_source = DVB_link
        self.WIFI_source = WIFI_source = WIFI_link
        self.delay1 = delay1 = 0
        self.delay2 = delay2 = 0
    
        ##################################################
        # Blocks
        ##################################################
        ############ source ##############################
        self.uhd_usrp_source_0 = uhd.usrp_source(
            ",".join(("addr=166.104.231.194", "")),
            uhd.stream_args(
                cpu_format="fc32",
                args='',
                channels=list(range(0,1)),
            ),
        )
        self.uhd_usrp_source_0.set_samp_rate(Rx_rate)
        self.uhd_usrp_source_0.set_center_freq(Rx_freq, 0)
        self.uhd_usrp_source_0.set_antenna("RX2", 0)
        self.uhd_usrp_source_0.set_gain(gain_value, 0)

        ########### Radar ################################
        self.uhd_usrp_sink_0 = uhd.usrp_sink(
            # ",".join(("addr=166.104.231.198", "")),
            ",".join(("addr=166.104.231.199", "")),
            uhd.stream_args(
                cpu_format="fc32",
                args='',
                channels=list(range(0,1)),
            ),
            '',
        )
        self.uhd_usrp_sink_0.set_samp_rate(Tx_rate1)
        self.uhd_usrp_sink_0.set_center_freq(Tx_freq1, 0)
        self.uhd_usrp_sink_0.set_antenna('TX/RX', 0)
        self.uhd_usrp_sink_0.set_gain(10, 0)

        ########### DVB-2T ################################
        self.uhd_usrp_sink_0_0 = uhd.usrp_sink(
            ",".join(("addr=166.104.231.199", "")),
            uhd.stream_args(
                cpu_format="fc32",
                args='',
                channels=list(range(0,1)),
            ),
            '',
        )

        self.uhd_usrp_sink_0_0.set_samp_rate(Tx_rate2)
        self.uhd_usrp_sink_0_0.set_center_freq(Tx_freq2, 0
                                               )
        self.uhd_usrp_sink_0_0.set_antenna('TX/RX', 0)
        self.uhd_usrp_sink_0_0.set_gain(10, 0)

        ########### WIFI ################################
        self.uhd_usrp_sink_0_1 = uhd.usrp_sink(
            # ",".join(("addr=166.104.231.184", "")),
            ",".join(("addr=166.104.231.199", "")),
            uhd.stream_args(
                cpu_format="fc32",
                args='',
                channels=list(range(0,1)),
            ),
            '',
        )
        self.uhd_usrp_sink_0_1.set_samp_rate(Tx_rate3)
        self.uhd_usrp_sink_0_1.set_center_freq(Tx_freq3, 0)
        self.uhd_usrp_sink_0_1.set_antenna('TX/RX', 0)
        self.uhd_usrp_sink_0_1.set_gain(10, 0)

        ##################################################
        self.hilbert_fc_0 = filter.hilbert_fc(50, window.WIN_BLACKMAN_hARRIS, 6.76)
        self.blocks_vco_f_0 = blocks.vco_f(8e5, 8e5*1.04, 1)
        self.blocks_multiply_xx_0 = blocks.multiply_vcc(1)
        self.analog_sig_source_x_0 = analog.sig_source_f(8e5, analog.GR_TRI_WAVE, 8e3, 1, 0, 0)
        self.blocks_vector_source_x_0_0 = blocks.vector_source_c(vector_source, False, 1, [])
        self.blocks_skiphead_0 = blocks.skiphead(gr.sizeof_gr_complex*1, skip_head)
        self.blocks_head_0 = blocks.head(gr.sizeof_gr_complex*1, head)
        self.blocks_file_source_0_0 = blocks.file_source(gr.sizeof_gr_complex*1, DVB_2T_source, False, 0, 0)
        self.blocks_file_source_0_0.set_begin_tag(pmt.PMT_NIL)
        self.blocks_file_source_0 = blocks.file_source(gr.sizeof_gr_complex*1, WIFI_source, False, 0, 0)
        self.blocks_file_source_0.set_begin_tag(pmt.PMT_NIL)
        self.blocks_file_sink_0 = blocks.file_sink(gr.sizeof_gr_complex*1, file_path, False)
        self.blocks_file_sink_0.set_unbuffered(False)
        self.blocks_delay_0 = blocks.delay(gr.sizeof_gr_complex*1, delay1)
        self.blocks_delay_0_0 = blocks.delay(gr.sizeof_gr_complex*1, delay2)
        self.root_raised_cosine_filter_1 = filter.fir_filter_ccf(
            1,
            firdes.root_raised_cosine(
                1,
                1e6,
                1e6/200,
                0.5,
                2))
        ##################################################
        # Connections
        
        ##################################################
        
        #radar
        if on_off_factor[0] == 1:
            self.connect((self.analog_sig_source_x_0, 0), (self.blocks_vco_f_0, 0))
            self.connect((self.blocks_vco_f_0, 0), (self.hilbert_fc_0, 0))
            self.connect((self.hilbert_fc_0, 0), (self.blocks_multiply_xx_0, 1))
            self.connect((self.blocks_vector_source_x_0_0, 0),(self.blocks_multiply_xx_0, 0)) 
            self.connect((self.blocks_multiply_xx_0, 0), (self.root_raised_cosine_filter_1, 0))
            self.connect((self.root_raised_cosine_filter_1, 0), (self.uhd_usrp_sink_0, 0))
    
          

        #dvb
        if on_off_factor[1] == 1:
            self.connect((self.blocks_file_source_0_0, 0),(self.root_raised_cosine_filter_1, 0))
            self.connect((self.root_raised_cosine_filter_1, 0),(self.blocks_delay_0, 0))
            self.connect((self.blocks_delay_0, 0),(self.uhd_usrp_sink_0_0, 0))

        #wifi
        if on_off_factor[2] == 1:
            self.connect((self.blocks_file_source_0, 0),(self.root_raised_cosine_filter_1, 0))
            self.connect((self.root_raised_cosine_filter_1, 0),(self.blocks_delay_0_0, 0),)
            self.connect((self.blocks_delay_0_0, 0),(self.uhd_usrp_sink_0_1, 0))
       
            
        #others
        self.connect((self.blocks_skiphead_0, 0), (self.blocks_head_0, 0))
        self.connect((self.blocks_head_0, 0), (self.blocks_file_sink_0, 0))
        self.connect((self.uhd_usrp_source_0, 0), (self.blocks_skiphead_0, 0))

    def get_delay1(self):
        return self.delay1

    def set_delay1(self,delay1):
        self.delay1 = delay1
        self.blocks_delay_0.set_dly(self.delay1)
        
    def get_delay2(self):
        return self.delay2

    def set_delay2(self,delay2):
        self.delay2 = delay2
        self.blocks_delay_0_0.set_dly(self.delay2)    
       
    def get_vector_source(self):
        return self.vector_source

    def set_vector_source(self,vector_source):
        self.vector_source = vector_source
        self.blocks_vector_source_x_0_0.set_data(self.vector_source)
    
    def get_DVB_2T_source(self):
        return self.DVB_2T_source

    def set_DVB_2T_source(self,DVB_2T_source):
        self.DVB_2T_source = DVB_2T_source
        self.blocks_file_source_0_0.open(DVB_2T_source, False, 0, 0)
        

    def get_WIFI_source(self):
        return self.WIFI_source

    def set_WIFI_source(self,WIFI_source):
        self.WIFI_source = WIFI_source
        self.blocks_file_source_0.open(WIFI_source, False, 0, 0)


    def get_offset1(self):
        return self.offset1

    def set_offset1(self, offset1):
        self.offset1 = offset1
        self.set_Tx_freq1(1.2e9 - 5e6 + self.offset1*1e6)

    def get_offset2(self):
        return self.offset2

    def set_offset2(self, offset2):
        self.offset2 = offset2    
        self.set_Tx_freq2(1.2e9 - 5e6 + self.offset2*1e6)

    def get_offset3(self):
        return self.offset3

    def set_offset3(self, offset3):
        self.offset3 = offset3    
        self.set_Tx_freq3(1.2e9 - 5e6 + self.offset3*1e6)

    def get_head(self):
        return self.head

    def set_head(self, head):
        self.head = head
        self.blocks_head_0.set_length(self.head)

    def set_skip_head(self, skip_head):
        self.skip_head = skip_head  

    def get_file_path(self):
        return self.file_path

    def set_file_path(self, file_path):
        self.file_path = file_path
        self.blocks_file_sink_0.open(self.file_path)

    def get_Tx_rate1(self):
        return self.Tx_rate1

    def set_Tx_rate1(self, Tx_rate1):
        self.Tx_rate1 = Tx_rate1
        self.uhd_usrp_sink_0.set_samp_rate(self.Tx_rate1)

    def get_Tx_rate2(self):
        return self.Tx_rate2

    def set_Tx_rate2(self, Tx_rate2):
        self.Tx_rate2 = Tx_rate2
        self.uhd_usrp_sink_0_0.set_samp_rate(self.Tx_rate2)

    def get_Tx_rate3(self):
        return self.Tx_rate3

    def set_Tx_rate3(self, Tx_rate3):
        self.Tx_rate3 = Tx_rate3
        self.uhd_usrp_sink_0_1.set_samp_rate(self.Tx_rate3)

    def get_Tx_freq1(self):
        return self.Tx_freq1

    def set_Tx_freq1(self, Tx_freq1):
        self.Tx_freq1 = Tx_freq1
        self.uhd_usrp_sink_0.set_center_freq(self.Tx_freq1, 0)

    def get_Tx_freq2(self):
        return self.Tx_freq2

    def set_Tx_freq2(self, Tx_freq2):
        self.Tx_freq2 = Tx_freq2
        self.uhd_usrp_sink_0_0.set_center_freq(self.Tx_freq2, 0)

    def get_Tx_freq3(self):
        return self.Tx_freq3

    def set_Tx_freq3(self, Tx_freq3):
        self.Tx_freq3 = Tx_freq3
        self.uhd_usrp_sink_0_1.set_center_freq(self.Tx_freq3, 0)

    def get_Rx_rate(self):
        return self.Rx_rate

    def set_Rx_rate(self, Rx_rate):
        self.Rx_rate = Rx_rate
        self.uhd_usrp_source_0.set_samp_rate(self.Rx_rate)

    def get_Rx_freq(self):
        return self.Rx_freq

    def set_Rx_freq(self, Rx_freq):
        self.Rx_freq = Rx_freq
        self.uhd_usrp_source_0.set_center_freq(self.Rx_freq, 0)

In [4]:
def sig_handler(sig=None, frame=None):
    tb.stop()
    tb.wait()

    sys.exit(0)

In [5]:
# random sub index and power setting
def make_ans_valid(n_ans, sub_idx_from, sub_idx_to):
    radar = []
    dvb = []
    wifi = []
    radar_real = []
    dvb_real = []
    wifi_real = []
    for i in range(n_ans):
        # n: Radar m: DVB-T2 k: WIFI
        if i < 200 or 600 <= i < 800:
            if i%5 == 0: 
                n = 3
                m = 2
            elif i%5 == 1: 
                n= 4
                m = 2
            elif i%5 == 2:
                n = 5
                m = 2
            elif i%5 == 3: 
                n = 3
                m = 10000
            elif i%5 == 4: 
                n = 4
                m = 2
        if 200 <= i < 400 or 800 <= i:
            if i%5 == 0: 
                n = 1
                m = 8
            elif i%5 == 1: 
                n= 3
                m = 10000
            elif i%5 == 2:
                n = 2
                m = 8
            elif i%5 == 3: 
                n = 1
                m = 10000
            elif i%5 == 4: 
                n = 2
                m = 8
        if 400 <= i < 600:
            if i%5 == 0: 
                n = 8
                m = 4
            elif i%5 == 1: 
                n= 8
                m = 10000
            elif i%5 == 2:
                n = 8
                m = 4
            elif i%5 == 3: 
                n = 9
                m = 4
            elif i%5 == 4: 
                n = 7
                m = 10000                 
        radar.append(n)
        radar_real.append(n)
        dvb.append(m)
        dvb_real.append(m)
        
    for i in range(n_ans):
            k = random.randint(sub_idx_from,sub_idx_to)
            if i%4 == 0:
                k = 10000
            elif i%4 ==1:
                while(1):
                    if k == radar[i] or k == radar[i+1] or k == radar[i+2] or k== dvb[i] or k== dvb[i+1] or k== dvb[i+2]:
                        k = random.randint(sub_idx_from,sub_idx_to)
                    else:
                        a = k
                        break
            else:
                k = a;
            wifi.append(k)
            wifi_real.append(k)
            
    for i in range(n_ans):
        if i%100 == 99:
            n = 4001
            m = 4002
            k = 4003
            radar[i] = n
            dvb[i] = m
            wifi[i] = k

    return radar, dvb, wifi, radar_real, dvb_real, wifi_real
   

In [6]:
# random sub index and power setting
def make_ans_test1(n_ans, sub_idx_from, sub_idx_to):
    radar = []
    dvb = []
    wifi = []
    radar_real = []
    dvb_real = []
    wifi_real = []
    for i in range(n_ans):
        # n: Radar m: DVB-T2 k: WIFI
        if i < 200 or 600 <= i < 800:
            if i%5 == 0: 
                n = 5
                m = 1
            elif i%5 == 1: 
                n= 6
                m = 1
            elif i%5 == 2:
                n = 4
                m = 10000
            elif i%5 == 3: 
                n = 5
                m = 1
            elif i%5 == 4: 
                n = 4
                m = 1
        if 200 <= i < 400 or 800 <= i:
            if i%5 == 0: 
                n = 7
                m = 10000
            elif i%5 == 1: 
                n= 6
                m = 8
            elif i%5 == 2:
                n = 5
                m = 10000
            elif i%5 == 3: 
                n = 6
                m = 8
            elif i%5 == 4: 
                n = 5
                m = 8
        if 400 <= i < 600:
            if i%5 == 0: 
                n = 2
                m = 10000
            elif i%5 == 1: 
                n= 2
                m = 5
            elif i%5 == 2:
                n = 3
                m = 5
            elif i%5 == 3: 
                n = 1
                m = 10000
            elif i%5 == 4: 
                n = 2
                m = 5                 
        radar.append(n)
        radar_real.append(n)
        dvb.append(m)
        dvb_real.append(m)
        
    for i in range(n_ans):
            k = random.randint(sub_idx_from,sub_idx_to)
            if i%4 == 0:
                k = 10000
            elif i%4 ==1:
                while(1):
                    if k == radar[i] or k == radar[i+1] or k == radar[i+2] or k== dvb[i] or k== dvb[i+1] or k== dvb[i+2]:
                        k = random.randint(sub_idx_from,sub_idx_to)
                    else:
                        a = k
                        break
            else:
                k = a;
            wifi.append(k)
            wifi_real.append(k)
            
    for i in range(n_ans):
        if i%100 == 99:
            n = 4001
            m = 4002
            k = 4003
            radar[i] = n
            dvb[i] = m
            wifi[i] = k

    return radar, dvb, wifi, radar_real, dvb_real, wifi_real

In [7]:
# random sub index and power setting
def make_ans_test2(n_ans, sub_idx_from, sub_idx_to):
    radar = []
    dvb = []
    wifi = []
    radar_real = []
    dvb_real = []
    wifi_real = []
    for i in range(n_ans):
        # n: Radar m: DVB-T2 k: WIFI
        if i < 200 or 600 <= i < 800:
            if i%5 == 0: 
                n = 5
                m = 1
            elif i%5 == 1: 
                n= 6
                m = 1
            elif i%5 == 2:
                n = 4
                m = 10000
            elif i%5 == 3: 
                n = 5
                m = 1
            elif i%5 == 4: 
                n = 4
                m = 1
        if 200 <= i < 400 or 800 <= i:
            if i%5 == 0: 
                n = 7
                m = 10000
            elif i%5 == 1: 
                n= 6
                m = 8
            elif i%5 == 2:
                n = 5
                m = 10000
            elif i%5 == 3: 
                n = 6
                m = 8
            elif i%5 == 4: 
                n = 5
                m = 8
        if 400 <= i < 600:
            if i%5 == 0: 
                n = 1
                m = 5
            elif i%5 == 1: 
                n= 3
                m = 5
            elif i%5 == 2:
                n = 1
                m = 5
            elif i%5 == 3: 
                n = 3
                m = 10000
            elif i%5 == 4: 
                n = 2
                m = 10000                 
        radar.append(n)
        radar_real.append(n)
        dvb.append(m)
        dvb_real.append(m)
        
    for i in range(n_ans):
            k = random.randint(sub_idx_from,sub_idx_to)
            if i%4 == 0:
                k = 10000
            elif i%4 ==1:
                while(1):
                    if k == radar[i] or k == radar[i+1] or k == radar[i+2] or k== dvb[i] or k== dvb[i+1] or k== dvb[i+2]:
                        k = random.randint(sub_idx_from,sub_idx_to)
                    else:
                        a = k
                        break
            else:
                k = a;
            wifi.append(k)
            wifi_real.append(k)
            
    for i in range(n_ans):
        if i%100 == 99:
            n = 4001
            m = 4002
            k = 4003
            radar[i] = n
            dvb[i] = m
            wifi[i] = k

    return radar, dvb, wifi, radar_real, dvb_real, wifi_real

In [8]:
# import numpy as np
# from sklearn import preprocessing

# radar, dvb, wifi, radar_real, dvb_real, wifi_real  = make_ans_test2(n_ans*10, sub_idx_from, sub_idx_to)
# for i in range(1,11):
#     temp_index = str(i)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/radar/radar_list'+temp_index+'.bin','wb') as f:
#         pickle.dump(radar[(i-1)*100:i*100],f)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/dvb/dvb_list'+temp_index+'.bin','wb') as f:
#         pickle.dump(dvb[(i-1)*100:i*100],f)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/wifi/wifi_list'+temp_index+'.bin','wb') as f:
#         pickle.dump(wifi[(i-1)*100:i*100],f)
        
        
# label_list = np.array([1,2,3,4,5,6,7,8,9])
# lb = preprocessing.LabelBinarizer()
# lb.fit(label_list)

# total_label = []
# total_label_real = []
# for i in range(1,11):
#     temp_index = str(i)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/radar/radar_list'+temp_index+'.bin','rb') as f:
#         radar = pickle.load(f)
#         radar_label = lb.transform(radar)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/dvb/dvb_list'+temp_index+'.bin','rb') as f:
#         dvb = pickle.load(f)
#         DVB_label = 2*lb.transform(dvb)
#     with open('/home/leesiho/matlab/lab/2/'+path+'/wifi/wifi_list'+temp_index+'.bin','rb') as f:
#         wifi = pickle.load(f)
#         WIFI_label = 3*lb.transform(wifi)
#     label = radar_label + DVB_label + WIFI_label
#     total_label.extend(label)
    
   
# radar_real_label = lb.transform(radar_real)
# DVB_real_label = 2*lb.transform(dvb_real)
# WIFI_real_label = 3*lb.transform(wifi_real)
# label_real = radar_real_label + DVB_real_label + WIFI_real_label
# total_label_real.extend(label_real)

    
# total_label = np.array(total_label)
# total_label_real = np.array(total_label_real)

# with open('/home/leesiho/matlab/lab/2/'+path+'/'+label_name,'wb') as f:
#     pickle.dump(total_label,f)
# with open('/home/leesiho/matlab/lab/2/'+path+'/real_label/'+label_name,'wb') as f:
#     pickle.dump(total_label_real,f)

In [9]:
for idx in range(2,3):
    temp_index = str(idx)
    tb = problem2()

    n_samples_for_slot = 30000 # 10k samples for 1 time slot / timeslot = 0.001 sec => 
    T_slot = 1/tb.get_Rx_rate() * n_samples_for_slot
    # print(T_slot)

    vector_source = Radar_vector
    DVB_2T_source = DVB_link
    WIFI_source = WIFI_link

    signal.signal(signal.SIGINT, sig_handler)
    signal.signal(signal.SIGTERM, sig_handler)

    #radar, dvb, wifi = make_ans(n_ans, sub_idx_from, sub_idx_to)
    with open('/home/leesiho/matlab/lab/2/'+path+'/radar/radar_list'+temp_index+'.bin','rb') as f:
        radar = pickle.load(f)
    with open('/home/leesiho/matlab/lab/2/'+path+'/dvb/dvb_list'+temp_index+'.bin','rb') as f:
        dvb = pickle.load(f)
    with open('/home/leesiho/matlab/lab/2/'+path+'/wifi/wifi_list'+temp_index+'.bin','rb') as f:
        wifi = pickle.load(f)
            
    # Throw away garbege data 
    temp_time = time.time()
    tb.set_head(0)
    tb.start()
    while(1):
        curr_time = time.time()
        if curr_time - temp_time >0.2:
            temp_time = curr_time
            break
    tb.stop()
    tb.wait()
    print('garbege data check = ',tb.blocks_file_sink_0.nitems_read(0))

    tb.set_skip_head(int(1e7))  

    for i in range(0, n_ans):

        if on_off_factor[0] == 1:
            tb.set_offset1(radar[i]) # radar
        if on_off_factor[1] == 1:
            tb.set_offset2(dvb[i]) # dvb
        if on_off_factor[2] == 1:
            tb.set_offset3(wifi[i]) # wifi
        
        tb.set_head((i+1)*n_samples_for_slot) 
        
        if on_off_factor[0] == 1:
            tb.set_vector_source(vector_source) # radar
        if on_off_factor[1] == 1:
            tb.set_DVB_2T_source(DVB_2T_source) # dvb
        if on_off_factor[2] == 1:
            tb.set_WIFI_source(WIFI_source) # wifi
        tb.start()            
        while(1):
            curr_time = time.time()
            if curr_time - temp_time > 0.05: #radar:0.1 dvb:0.1 wifi:0.1
                temp_time = curr_time
                break
        print(str(i)+'th data =',tb.blocks_file_sink_0.nitems_read(0))
        tb.wait()
        tb.stop()


[INFO] [UHD] linux; GNU C++ version 9.3.0; Boost_107100; UHD_3.15.0.0-release
[INFO] [USRP2] Opening a USRP2/N-Series device...
[INFO] [USRP2] Current recv frame size: 1472 bytes
[INFO] [USRP2] Current send frame size: 1472 bytes
[WARNING] [UDP] The send buffer could not be resized sufficiently.
Target sock buff size: 2500000 bytes.
Actual sock buff size: 1048576 bytes.
See the transport application notes on buffer resizing.
Please run: sudo sysctl -w net.core.wmem_max=2500000
[WARNING] [UDP] The send buffer could not be resized sufficiently.
Target sock buff size: 2500000 bytes.
Actual sock buff size: 1048576 bytes.
See the transport application notes on buffer resizing.
Please run: sudo sysctl -w net.core.wmem_max=2500000
[WARNING] [UDP] The send buffer could not be resized sufficiently.
Target sock buff size: 2500000 bytes.
Actual sock buff size: 1048576 bytes.
See the transport application notes on buffer resizing.
Please run: sudo sysctl -w net.core.wmem_max=2500000
[WARNING] [UHD

garbege data check =  0
0th data = 30000
1th data = 30000
2th data = 30000
3th data = 30000
4th data = 30000
5th data = 30000
6th data = 30000
7th data = 30000
8th data = 30000
9th data = 30000
10th data = 30000
11th data = 30000
12th data = 30000
13th data = 30000
14th data = 30000
15th data = 30000
16th data = 30000
17th data = 30000
18th data = 30000
19th data = 30000
20th data = 30000
21th data = 30000
22th data = 30000
23th data = 30000
24th data = 30000
25th data = 30000
26th data = 30000
27th data = 30000
28th data = 30000
29th data = 30000
30th data = 30000
31th data = 30000
32th data = 30000
33th data = 30000
34th data = 30000
35th data = 30000
36th data = 30000
37th data = 30000
38th data = 30000
39th data = 30000
40th data = 30000
41th data = 30000
42th data = 30000
43th data = 30000
44th data = 30000
45th data = 30000
46th data = 30000
47th data = 30000
48th data = 30000
49th data = 30000
50th data = 30000
51th data = 30000
52th data = 30000
53th data = 30000
54th data = 30